In [21]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
import json


In [ ]:
# Đọc dữ liệu đã xử lý
data = pd.read_csv('../data/processed/diamonds_clean.csv')

# Đọc thông tin folds
with open('../data/kfold_indices.json', 'r') as f:
    fold_indices = json.load(f)


✅ Đọc 5 fold thành công!


In [23]:
# Giả sử cột target là 'price'
X = data.drop(columns=['price'])
y = data['price']

print("Shape dữ liệu:", X.shape, y.shape)


Shape dữ liệu: (53794, 9) (53794,)


In [24]:
def preprocess(X):
    ones = np.ones((X.shape[0], 1))
    return np.hstack((ones, X))


In [25]:
class OLSLinearRegression:
    def fit(self, X, y):
        X_pinv = np.linalg.pinv(X)
        self.w = X_pinv @ y
        return self

    def get_params(self):
        return self.w

    def predict(self, X):
        return X @ self.w

In [26]:
def mae(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))

In [27]:
def cross_val_mae_custom(X, y, model_class, k=5, random_state=42):
    kf = KFold(n_splits=k, shuffle=True, random_state=random_state)
    maes = []
    
    for train_idx, val_idx in kf.split(X):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        
        X_train_proc = preprocess(X_train.values)
        X_val_proc = preprocess(X_val.values)
        
        model = model_class()
        model.fit(X_train_proc, y_train.values)
        y_pred = model.predict(X_val_proc)
        
        maes.append(mae(y_val.values, y_pred))
    
    return np.mean(maes)

In [28]:
def cross_val_mae_sklearn(X, y, model, k=5, random_state=42):
    kf = KFold(n_splits=k, shuffle=True, random_state=random_state)
    maes = []
    
    for train_idx, val_idx in kf.split(X):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        maes.append(mean_absolute_error(y_val, y_pred))
    
    return np.mean(maes)

In [ ]:
model_definitions = [
    # Minh
    {"Tên mô hình": "Model 1: carat", 
     "Đặc trưng": ['carat']},

    # My
    {"Tên mô hình": "Model 2: carat + depth + table", 
     "Đặc trưng": ['carat', 'depth', 'table']},

    # Kiệt
    {"Tên mô hình": "Model 3: carat + x + y + z", 
     "Đặc trưng": ['carat', 'x', 'y', 'z']},

    # Thảo 
    {"Tên mô hình": "Model 4: carat + depth + table + x + y + z", 
     "Đặc trưng": ['carat', 'depth', 'table', 'x', 'y', 'z']}

    # Duy
    {"Tên mô hình": "Model 4: carat + depth + table + x + y + z", 
     "Đặc trưng": ['carat', 'depth', 'table', 'x', 'y', 'z']}
]


In [30]:
results = []

# --- Custom model (OLS) ---
for m in model_definitions:
    cols = m["Đặc trưng"]
    mae_score = cross_val_mae_custom(X[cols], y, OLSLinearRegression, k=5)
    results.append({
        "Tên mô hình": m["Tên mô hình"],
        "Đặc trưng": cols,
        "Loại mô hình": "Custom OLS",
        "MAE trung bình": mae_score
    })

# --- Baseline: sklearn LinearRegression ---
baseline_model = LinearRegression()
baseline_mae = cross_val_mae_sklearn(X, y, baseline_model, k=5)

results.append({
    "Tên mô hình": "Baseline: sklearn LinearRegression (tất cả features)",
    "Đặc trưng": list(X.columns),
    "Loại mô hình": "sklearn",
    "MAE trung bình": baseline_mae
})

results_df = pd.DataFrame(results)
results_df.sort_values("MAE trung bình")

,Tên mô hình,Đặc trưng,Loại mô hình,MAE trung bình
4,Baseline: sklearn LinearRegression (tất cả fea...,"[carat, cut, color, clarity, depth, table, x, ...",sklearn,804.458759
3,Model 4: carat + depth + table + x + y + z,"[carat, depth, table, x, y, z]",Custom OLS,888.079178
2,Model 3: carat + x + y + z,"[carat, x, y, z]",Custom OLS,905.567051
1,Model 2: carat + depth + table,"[carat, depth, table]",Custom OLS,994.625593
0,Model 1: carat,[carat],Custom OLS,1007.743775
